In [ ]:
import numpy as np

class RNN:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights_z = np.random.randn(hidden_size, hidden_size + input_size) / np.sqrt(hidden_size + input_size)
        self.weights_r = np.random.randn(hidden_size, hidden_size + input_size) / np.sqrt(hidden_size + input_size)
        self.weights_h = np.random.randn(hidden_size, hidden_size + input_size) / np.sqrt(hidden_size + input_size)
        self.bias_z = np.zeros((hidden_size, 1))
        self.bias_r = np.zeros((hidden_size, 1))
        self.bias_h = np.zeros((hidden_size, 1))

    def forward_propagation(self, inp_t, prev_h):
        self.inp_t = inp_t
        self.prev_h = prev_h
        z = np.concatenate((self.prev_h, self.inp_t), axis = 0)
        ug = self.sigmoid(np.dot(self.weights_z, z) + self.bias_z)
        rg = self.sigmoid(np.dot(self.weights_r, z) + self.bias_r)
        c_hid = np.tanh(np.dot(self.weights_h, z) + self.bias_h)
        h_t = (1 - ug) * prev_h + ug * c_hid
        self.prev_h = h_t 
        return h_t

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def train(self, inputs, targets, learning_rate = 0.1, num_epochs = 100, gradient_clip = 1.0, learning_rate_decay = 0.1):
        for epoch in range(num_epochs):
            loss = 0
            h_prev = np.zeros((self.hidden_size, 1))
            for t in range(len(inputs)):
                x_t = inputs[t]
                y_t = targets[t]
                h_t = self.forward_propagation(x_t, h_prev)
                loss += np.sum(np.square(y_t - h_t))
                delta_t = -2 * (y_t - h_t)
                dz = delta_t * h_t * (1 - h_t)
                dw_z = np.outer(dz, np.concatenate((h_prev, x_t)))
                self.weights_z -= learning_rate * np.clip(dw_z, -gradient_clip, gradient_clip)
                self.bias_z -= learning_rate * np.clip(dz, -gradient_clip, gradient_clip)

                dr = np.dot(self.weights_h.T, dz) * h_prev * (1 - h_prev)
                dw_r = np.outer(dr, np.concatenate((h_prev, x_t)))
                self.weights_r -= learning_rate * np.clip(dw_r, -gradient_clip, gradient_clip)
                self.bias_r -= learning_rate * np.clip(dr, -gradient_clip, gradient_clip)

                dh = np.dot(self.weights_r.T, dr) + np.dot(self.weights_z.T, dz)
                dc_hid = dh * h_prev * (1 - np.square(np.tanh(np.dot(self.weights_h, np.concatenate((h_prev, x_t))))))

                dw_h = np.outer(dc_hid, np.concatenate((h_prev, x_t)))
                self.weights_h -= learning_rate * np.clip(dw_h, -gradient_clip, gradient_clip)
                self.bias_h -= learning_rate * np.clip(dc_hid, -gradient_clip, gradient_clip)

                h_prev = h_t
            learning_rate *= (1.0 / (1.0 + learning_rate_decay * epoch))
            
            print(f"Epoch: {epoch + 1}, Loss: {loss}")

In [ ]:
with open('C:/Users/ASUS/OneDrive - Ayb Educational Foundation/Desktop/lotr.txt', 'r') as file:
    text = file.read()

In [ ]:
import re

text = text.lower()
text = re.sub(r'[^a-z\s]', '', text)

In [ ]:
sequence_length = 10

inputs = []
targets = []

for i in range(len(text) - sequence_length):
    inputs.append(text[i:i + sequence_length])
    targets.append(text[i + sequence_length])

In [ ]:
characters = sorted(list(set(text)))
input_size = len(characters)

In [ ]:
char_to_index = {char: i for i, char in enumerate(characters)}
index_to_char = {i: char for i, char in enumerate(characters)}

In [ ]:
inputs_numeric = np.zeros((len(inputs), sequence_length, input_size), dtype = np.float32)
targets_numeric = np.zeros((len(targets), input_size), dtype = np.float32)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(inputs, targets)):
    for t, char in enumerate(input_text):
        inputs_numeric[i, t, char_to_index[char]] = 1.0
    targets_numeric[i, char_to_index[target_text]] = 1.0

In [ ]:
rnn = RNN(input_size, 256)
rnn.train(inputs_numeric, targets_numeric, learning_rate = 0.1, num_epochs = 100)